In [244]:
import requests
import pandas as pd
import plotly.express as px
import plotly
colors = px.colors.qualitative.D3 + px.colors.qualitative.Dark24 + plotly.colors.DEFAULT_PLOTLY_COLORS + px.colors.qualitative.Plotly + px.colors.qualitative.Alphabet

In [183]:
def download_data():
    url_metadata = "https://www.data.gouv.fr/fr/organizations/sante-publique-france/datasets-resources.csv"
    metadata = requests.get(url_metadata)
    
    with open('data/france/metadata.csv', 'wb') as f:
        f.write(metadata.content)
        
    df_metadata = pd.read_csv('data/france/metadata.csv', sep=";")
    url_data = df_metadata[df_metadata['url'].str.contains("/donnees-hospitalieres-covid19")]["url"].values[0]
    
    data = requests.get(url_data)
    with open('data/france/donnes-hospitalieres-covid19.csv', 'wb') as f:
        f.write(data.content)

download_data()

In [272]:
def import_data():
    df = pd.read_csv('data/france/donnes-hospitalieres-covid19.csv', sep=";")
    df_regions = pd.read_csv('data/france/departments_regions_france_2016.csv', sep=",")
    df_pop = pd.read_csv('data/france/population_regions.csv', sep=",")
    df = df.merge(df_regions, left_on='dep', right_on='departmentCode')
    df = df.merge(df_pop, left_on='regionName', right_on='regionName')
    
    df['rea_pop'] = df['rea']/df['regionPopulation']*100000
    df['rad_pop'] = df['rad']/df['regionPopulation']*100000
    df['dc_pop'] = df['dc']/df['regionPopulation']*100000
    df['hosp_pop'] = df['hosp']/df['regionPopulation']*100000
    return df
df = import_data()
df_region = df[df["sexe"] == 0]
df_region = df_region.groupby(['regionName', 'jour']).sum().reset_index()
df_region
df_region[df_region["regionName"]=="Ile-de-France"]
#df_region["dc_cumsum"] = df.groupby(by=['regionName','jour']).sum().groupby(level=[0]).cumsum().reset_index()["dc"]

df_region

,regionName,jour,sexe,hosp,rea,rad,dc,regionCode,regionPopulation,rea_pop,rad_pop,dc_pop,hosp_pop
0,Auvergne-Rhône-Alpes,2020-03-18,0,172,35,69,7,1008.0,49568628,0.847310,1.670411,0.169462,4.163924
1,Auvergne-Rhône-Alpes,2020-03-19,0,385,79,132,22,1008.0,49568628,1.912500,3.195570,0.532595,9.320411
2,Auvergne-Rhône-Alpes,2020-03-20,0,463,87,179,33,1008.0,49568628,2.106171,4.333386,0.798892,11.208702
3,Auvergne-Rhône-Alpes,2020-03-21,0,525,88,192,38,1008.0,49568628,2.130380,4.648101,0.919937,12.709652
4,Auvergne-Rhône-Alpes,2020-03-22,0,641,106,212,47,1008.0,49568628,2.566139,5.132278,1.137816,15.517880
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Provence-Alpes-Côte d'Azur,2020-03-21,0,326,66,310,13,558.0,15838038,2.500310,11.743879,0.492485,12.350015
140,Provence-Alpes-Côte d'Azur,2020-03-22,0,388,78,327,14,558.0,15838038,2.954911,12.387898,0.530369,14.698790
141,Provence-Alpes-Côte d'Azur,2020-03-23,0,481,112,377,20,558.0,15838038,4.242950,14.282072,0.757670,18.221954
142,Provence-Alpes-Côte d'Azur,2020-03-24,0,545,133,435,26,558.0,15838038,5.038503,16.479314,0.984970,20.646497


In [273]:
fig = px.line(x=df_region['jour'], y=df_region['rea']+df_region['hosp'], color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title_text='Nombre de patients hospitalisés (dont réa.)'
)

fig.show()

In [274]:
fig = px.line(x=df_region['jour'], y=df_region['rea_pop']+df_region['hosp_pop'], color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title_text='Nombre de patients hospitalisés (dont réa.) pour 100k habitants'
)
fig.show()

In [285]:

fig = px.line(x=df_region['jour'], y=df_region['rea_pop']+df_region['hosp_pop'], color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title_text='Nombre de patients hospitalisés (dont réa.) pour 100k habitants de chaque région'
)
fig.show()

In [277]:
fig = px.line(x=df_region['jour'], y=df_region['dc'], color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title_text='Nombre de décès cumulés'
)
fig.show()

In [279]:
fig = px.line(x=df_region['jour'], y = df_region['dc']/df_region['regionPopulation']*100000, color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title_text='Nombre de décès cumulés pour 100k habitants de chaque région'
)
fig.show()

In [282]:
fig = px.bar(x=df_region['jour'], y = df_region['dc'], color=df_region["regionName"], color_discrete_sequence=colors)
fig.update_layout(
    title_text='Nombre de décès journaliers'
)
fig.show()
df_region[df_region["regionName"] == "Auvergne-Rhône-Alpes"]

,regionName,jour,sexe,hosp,rea,rad,dc,regionCode,regionPopulation,rea_pop,rad_pop,dc_pop,hosp_pop
0,Auvergne-Rhône-Alpes,2020-03-18,0,172,35,69,7,1008.0,49568628,0.847310,1.670411,0.169462,4.163924
1,Auvergne-Rhône-Alpes,2020-03-19,0,385,79,132,22,1008.0,49568628,1.912500,3.195570,0.532595,9.320411
2,Auvergne-Rhône-Alpes,2020-03-20,0,463,87,179,33,1008.0,49568628,2.106171,4.333386,0.798892,11.208702
3,Auvergne-Rhône-Alpes,2020-03-21,0,525,88,192,38,1008.0,49568628,2.130380,4.648101,0.919937,12.709652
4,Auvergne-Rhône-Alpes,2020-03-22,0,641,106,212,47,1008.0,49568628,2.566139,5.132278,1.137816,15.517880
5,Auvergne-Rhône-Alpes,2020-03-23,0,853,157,254,70,1008.0,49568628,3.800791,6.149051,1.694620,20.650158
6,Auvergne-Rhône-Alpes,2020-03-24,0,981,190,314,86,1008.0,49568628,4.599683,7.601582,2.081962,23.748892
7,Auvergne-Rhône-Alpes,2020-03-25,0,1230,258,404,107,1008.0,49568628,6.245886,9.780380,2.590348,29.776898


In [283]:
df_region3 = df_region.groupby("regionName").sum().reset_index()
fig = px.bar(x=df_region3['regionName'], y = df_region3['dc'], color_discrete_sequence=colors).update_xaxes(categoryorder="total descending")
fig.update_layout(
    title_text='Nombre de décès cumulés'
)
fig.show()

In [161]:
df_bar = pd.melt(df, id_vars=['jour', 'regionName'], value_vars=['rea', 'hosp'])
df_bar = df_bar.groupby(['jour', 'variable']).sum().reset_index()
fig = px.bar(df_bar, x='jour', y='value', color="variable")
fig.show()

In [162]:
df_bar = pd.melt(df, id_vars=['jour', 'regionName'], value_vars=['rea_pop', 'hosp_pop'])
df_bar = df_bar.groupby(['regionName', 'variable']).sum().reset_index()

fig = px.bar(df_bar, x='regionName', y='value', color="variable")
fig.show()
df_bar

xs = df_bar['regionName'].values
ys = df_bar['value'].values
color = df_bar['variable'].values

import chart_studio.plotly as py
import plotly.graph_objs as go

xs = ['giraffes', 'orangutans', 'monkeys']
ys = [20, 14, 23]
data = [go.Bar(
    x=xs,
    y=ys,
    marker={
        'color': ys,
        #'colorscale': 'Viridis'
    }
)]

layout = {
    'xaxis': {
        'categoryorder': 'array',
        'categoryarray': [x for _, x in sorted(zip(color, xs))]
    }
}

fig = go.FigureWidget(data=data, layout=layout)
fig

FigureWidget({
    'data': [{'marker': {'color': [20, 14, 23]},
              'type': 'bar',
              'ui…

In [163]:

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack'
)

x=['b', 'a', 'c', 'd']
fig = go.Figure(go.Bar(x=x, y=[2,5,1,9], name='Montreal'))
fig.add_trace(go.Bar(x=x, y=[1, 4, 9, 16], name='Ottawa'))
fig.add_trace(go.Bar(x=x, y=[6, 8, 4.5, 8], name='Toronto'))

fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig.show()

NameError: name 'trace1' is not defined

In [284]:
last_day = df['jour'].max()
df_bar = pd.melt(df, id_vars=['jour','regionName'], value_vars=['rea_pop', 'hosp_pop'])
df_bar = df_bar[df_bar['jour']==last_day]
df_bar = df_bar.groupby(['regionName', 'variable']).sum().reset_index()

fig = px.bar(df_bar, x='regionName', y='value', color="variable").update_xaxes(categoryorder="total descending")

fig.update_layout(
    title_text='Nombre de patients hospitalisés et en réanimation pour 100k hab le 25 mars'
)
fig.data[-1].name = 'Réanimation'
fig.data[-2].name = 'Hospitalisation'

fig.show()
df_bar

,regionName,variable,value
0,Auvergne-Rhône-Alpes,hosp_pop,58.585442
1,Auvergne-Rhône-Alpes,rea_pop,12.201266
2,Bourgogne-Franche-Comté,hosp_pop,82.592726
3,Bourgogne-Franche-Comté,rea_pop,20.980371
4,Bretagne,hosp_pop,18.384000
5,Bretagne,rea_pop,4.421468
6,Centre,hosp_pop,24.769631
7,Centre,rea_pop,8.786780
8,Corse,hosp_pop,91.141804
9,Corse,rea_pop,10.126867
